# Example data anonymization

In Pega CDH 8.5 and up, it's now possible to record the historical data as seen by the Adaptive Models. See [this academy challenge](https://academy.pega.com/challenge/exporting-historical-data/v4) for reference. This historical data can be further used to experiment with offline models, but also to fine-tune the OOTB Gradient Boosting model. However, sharing this information with Pega can be sensitive as it contains raw predictor data. 

To this end, we provide a simple and transparent script to fully anonimize this dataset.

The DataAnonymization script is now part of pdstools, and you can import it directly as such.

In [1]:
# These lines are only for rendering in the docs, and are hidden through Jupyter tags
# Do not run if you're running the notebook seperately

import os  
import sys
import plotly.io as pio
pio.renderers.default = "notebook_connected"

sys.path.append("../../../")
sys.path.append('../../python')

In [2]:
from pdstools import ADMDatamart
from pdstools import Config, DataAnonymization
import polars as pl

## Input data

To demonstrate this process, we're going to anonymise this toy example dataframe:

In [3]:
pl.read_ndjson('../../../../data/SampleHDS.json')

Context_Name,Customer_MaritalStatus,Customer_CLV,Customer_City,IH_Web_Inbound_Accepted_pxLastGroupID,Decision_Outcome
str,str,i64,str,str,str
"""FirstMortgage3…","""Married""",1460,"""Port Raoul""","""Account""","""Rejected"""
"""FirstMortgage3…","""Unknown""",669,"""Laurianneshire…","""AutoLoans""","""Accepted"""
"""MoneyMarketSav…","""No Resp+""",1174,"""Jacobshaven""","""Account""","""Rejected"""
"""BasicChecking""","""Unknown""",1476,"""Lindton""","""Account""","""Rejected"""
"""BasicChecking""","""Married""",1211,"""South Jimmiesh…","""DepositAccount…","""Accepted"""
"""UPlusFinPerson…","""No Resp+""",533,"""Bergeville""",null,"""Rejected"""
"""BasicChecking""","""No Resp+""",555,"""Willyville""","""DepositAccount…","""Rejected"""


As you can see, this dataset consists of regular predictors, IH predictors, context keys and the outcome column. Additionally, some columns are numeric, others are strings. Let's first initialize the DataAnonymization class.

In [4]:
anon = DataAnonymization(hds_folder='../../../../data/')

By default, the class applies a set of anonymisation techniques:
- Column names are remapped to a non-descriptive name
- Categorical values are hashed with a random seed
- Numerical values are normalized between 0 and 1
- Outcomes are mapped to a binary outcome.

To apply these techniques, simply call `.process()`:

In [5]:
anon.process()

PREDICTOR_0,filename,PREDICTOR_2,PREDICTOR_3,Context_Name,IH_PREDICTOR_0,Decision_Outcome
f64,str,str,str,str,str,bool
1.2927e19,"""../../../../da…","""10727426233356…","""19827706664672…","""70215499258474…","""77158594327606…",false
1.4856e19,"""../../../../da…","""16189188749321…","""16477506493013…","""70215499258474…","""13903935423131…",true
5.6458e17,"""../../../../da…","""15780810512524…","""82662990409455…","""18611825150735…","""77158594327606…",false
4.0723e18,"""../../../../da…","""16189188749321…","""15395893331721…","""15344984046735…","""77158594327606…",false
1.4677e19,"""../../../../da…","""10727426233356…","""45868147790892…","""15344984046735…","""17910331925401…",true
1.6330e19,"""../../../../da…","""15780810512524…","""68428554856256…","""98838480582218…",null,false
8.7675e18,"""../../../../da…","""15780810512524…","""73294535464203…","""15344984046735…","""17910331925401…",false


To trace back the columns to their original names, the class also contains a mapping, which does not have to be provided.

In [6]:
anon.column_mapping

{'Customer_CLV': 'PREDICTOR_0',
 'filename': 'filename',
 'Customer_MaritalStatus': 'PREDICTOR_2',
 'Customer_City': 'PREDICTOR_3',
 'Context_Name': 'Context_Name',
 'IH_Web_Inbound_Accepted_pxLastGroupID': 'IH_PREDICTOR_0',
 'Decision_Outcome': 'Decision_Outcome'}

## Configs

Each capability can optionally be turned off - see below for the full list of config options, and refer to the API reference for the full description.

In [7]:
dict(zip(Config.__init__.__code__.co_varnames[1:], Config.__init__.__defaults__))

{'config_file': None,
 'hds_folder': '.',
 'use_datamart': False,
 'datamart_folder': 'datamart',
 'output_format': 'ndjson',
 'output_folder': 'output',
 'mapping_file': 'mapping.map',
 'mask_predictor_names': True,
 'mask_context_key_names': False,
 'mask_ih_names': True,
 'mask_outcome_name': False,
 'mask_predictor_values': True,
 'mask_context_key_values': True,
 'mask_ih_values': True,
 'mask_outcome_values': True,
 'context_key_label': 'Context_*',
 'ih_label': 'IH_*',
 'outcome_column': 'Decision_Outcome',
 'positive_outcomes': ['Accepted', 'Clicked'],
 'negative_outcomes': ['Rejected', 'Impression'],
 'special_predictors': ['Decision_DecisionTime',
  'Decision_OutcomeTime',
  'Decision_Rank'],
 'sample_percentage_schema_inferencing': 0.01}

It's easy to change these parameters by just passing the keyword arguments. In the following example, we
- Keep the IH predictor names
- Keep the outcome values
- Keep the context key values
- Keep the context key predictor names

In [8]:
anon = DataAnonymization(
    hds_folder="../../../../data/",
    mask_ih_names=False,
    mask_outcome_values=False,
    mask_context_key_values=False,
    mask_context_key_names=False,
)
anon.process()


PREDICTOR_0,filename,PREDICTOR_2,PREDICTOR_3,Context_Name,IH_Web_Inbound_Accepted_pxLastGroupID,Decision_Outcome
f64,str,str,str,str,str,str
1.2927e19,"""../../../../da…","""36124973607757…","""72711084108268…","""FirstMortgage3…","""14535683641559…","""Rejected"""
1.4856e19,"""../../../../da…","""58910999596412…","""97325876408744…","""FirstMortgage3…","""20517984913660…","""Accepted"""
5.6458e17,"""../../../../da…","""17406318975975…","""10404237830653…","""MoneyMarketSav…","""14535683641559…","""Rejected"""
4.0723e18,"""../../../../da…","""58910999596412…","""17182115159055…","""BasicChecking""","""14535683641559…","""Rejected"""
1.4677e19,"""../../../../da…","""36124973607757…","""15523002767425…","""BasicChecking""","""35252292059928…","""Accepted"""
1.6330e19,"""../../../../da…","""17406318975975…","""69593937093040…","""UPlusFinPerson…",null,"""Rejected"""
8.7675e18,"""../../../../da…","""17406318975975…","""15882842710381…","""BasicChecking""","""35252292059928…","""Rejected"""


The configs can also be written and read as such:

In [9]:
anon.config.save_to_config_file('config.json')

In [10]:
anon = DataAnonymization(config=Config(config_file='config.json'))
anon.process()

PREDICTOR_0,filename,PREDICTOR_2,PREDICTOR_3,Context_Name,IH_Web_Inbound_Accepted_pxLastGroupID,Decision_Outcome
f64,str,str,str,str,str,str
1.2927e19,"""../../../../da…","""17491073916777…","""41896191566243…","""FirstMortgage3…","""14961785151035…","""Rejected"""
1.4856e19,"""../../../../da…","""13958276875042…","""22390569062254…","""FirstMortgage3…","""11133130621742…","""Accepted"""
5.6458e17,"""../../../../da…","""11110908762850…","""45198827745695…","""MoneyMarketSav…","""14961785151035…","""Rejected"""
4.0723e18,"""../../../../da…","""13958276875042…","""76763848269141…","""BasicChecking""","""14961785151035…","""Rejected"""
1.4677e19,"""../../../../da…","""17491073916777…","""78333156419768…","""BasicChecking""","""13887624510702…","""Accepted"""
1.6330e19,"""../../../../da…","""11110908762850…","""45473675247257…","""UPlusFinPerson…",null,"""Rejected"""
8.7675e18,"""../../../../da…","""11110908762850…","""57645686802200…","""BasicChecking""","""13887624510702…","""Rejected"""


## Exporting
Two functions export:
- `create_mapping_file()` writes the mapping file of the predictor names
- `write_to_output()` writes the processed dataframe to disk

Write to output accepts the following extensions: `["ndjson", "parquet", "arrow", "csv"]`

In [11]:
anon.create_mapping_file()
with open('mapping.map') as f:
    print(f.read())

Customer_CLV=PREDICTOR_0
filename=filename
Customer_MaritalStatus=PREDICTOR_2
Customer_City=PREDICTOR_3
Context_Name=Context_Name
IH_Web_Inbound_Accepted_pxLastGroupID=IH_Web_Inbound_Accepted_pxLastGroupID
Decision_Outcome=Decision_Outcome



In [12]:
anon.write_to_output(ext='arrow')

In [13]:
pl.read_ipc('output/hds.arrow')

PREDICTOR_0,PREDICTOR_2,PREDICTOR_3,Context_Name,IH_Web_Inbound_Accepted_pxLastGroupID,Decision_Outcome
f64,str,str,str,str,str
1.2927e19,"""81701425091879…","""16738698705610…","""FirstMortgage3…","""15787035923095…","""Rejected"""
1.4856e19,"""17916719250824…","""25888401162888…","""FirstMortgage3…","""47095074431422…","""Accepted"""
5.6458e17,"""46310374144028…","""64556645052171…","""MoneyMarketSav…","""15787035923095…","""Rejected"""
4.0723e18,"""17916719250824…","""14486510871865…","""BasicChecking""","""15787035923095…","""Rejected"""
1.4677e19,"""81701425091879…","""14632552642576…","""BasicChecking""","""13531362754299…","""Accepted"""
1.6330e19,"""46310374144028…","""55197846333405…","""UPlusFinPerson…",null,"""Rejected"""
8.7675e18,"""46310374144028…","""16018062041125…","""BasicChecking""","""13531362754299…","""Rejected"""


## Advanced: Hash fuctions

By default, we use [the same hashing algorithm Polars](https://pola-rs.github.io/polars/py-polars/html/reference/expressions/api/polars.Expr.hash.html#polars.Expr.hash) uses: [xxhash](https://github.com/Cyan4973/xxHash), as implemented [here](https://github.com/pola-rs/polars/blob/3f287f370b3c388ed2f3f218b2c096382548136f/polars/polars-core/src/vector_hasher.rs#L266). xxhash is fast to compute, and you can check its performance in collision, dispersion and randomness [here](https://github.com/Cyan4973/xxHash/tree/dev/tests). 

xxhash accepts four distinct seeds, but by default we set the seeds to `0`. It is possible to set the `seed` argument of the `process()` function to `'random'`, which will set all four seeds to a random integer between `0` and `1000000000`. Alternatively, it is possible to supply the four seeds manually with arguments `seed`, `seed_1`, `seed_2` and `seed_3`. 

If the xxhash with (random) seed(s) is not deemed sufficiently secure, it is possible to use your own hashing algorithm.

Note that since we're now running python code and not native Polars code anymore, this will be _significantly_ slower. Nonetheless, it is possible.

Just as an example - this is how one would use sha3_256:

In [14]:
from hashlib import sha3_256

anon.process(algorithm=lambda x: sha3_256(x.encode()).hexdigest())

ComputeError: AttributeError: 'int' object has no attribute 'encode'